# Similarity with fasttext

### Load model

In [2]:
import csv

import fasttext
import fasttext.util
import pandas as pd
#fasttext.util.download_model('nl', if_exists='ignore')  # Dutch

In [2]:
ft = fasttext.load_model('cc.nl.300.bin')

### Load train and test df

In [50]:
# Train
train = pd.read_csv("result.csv")
train.drop(columns={'label'}, inplace = True)
train.dropna(0, subset=["text_clean", "sentiment"], inplace = True)
cleanup_label = {"sentiment": {"Yes": 1, "No": 0}}
train.replace(cleanup_label, inplace=True)
train.rename(columns={"sentiment": "label"}, inplace=True)
train = train[train["type"] != "title"].copy()

In [81]:
# Test
# Load the previous dataset for testing
test = pd.read_csv("../data/processed/selected_articles/1990s/to_label_gas.csv")
test["label"] = None
# Drop labeled articles
test = test.loc[~test.index_article.isin(train.index_article)].copy()
# Drop na and titles
test.dropna(0, subset=["text_clean"], inplace = True)
test = test[test["type"] != "title"].copy()

42816
42314
41257


### Create train valid sets

In [85]:
# Split train and validation
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(train["text_clean"], train["label"],
                                                  stratify=train["label"],
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

### Create train and valid adding Over/Under sampling

In [103]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# define oversampling strategy
over = RandomOverSampler(sampling_strategy=0.5)
# fit and apply the transform
X, y = over.fit_resample(train[["text_clean"]], train[["label"]])
# define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.5)
# fit and apply the transform
X, y = under.fit_resample(X, y)

In [104]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(X, y,
                                                  stratify=y,
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

### Create txt for fasttext

In [90]:
file = open('gas.1990.train', 'w+')
for idx, row in train_x.items():
    line = '__label__' + str(train_y[idx]) + ' ' + train_x[idx]
    file.write(line + '\n')
    
file = open('gas.1990.valid', 'w+')
for idx, row in valid_x.items():
    line = '__label__' + str(valid_y[idx]) + ' ' + valid_x[idx]
    file.write(line + '\n')

In [105]:
file = open('gas.1990.train.ou', 'w+')
for idx, row in train_x.items():
    line = '__label__' + str(train_y[idx]) + ' ' + train_x[idx]
    file.write(line + '\n')
    
file = open('gas.1990.valid.ou', 'w+')
for idx, row in valid_x.items():
    line = '__label__' + str(valid_y[idx]) + ' ' + valid_x[idx]
    file.write(line + '\n')

KeyError: 'text_clean'

### Train fasttext model

In [53]:
model = fasttext.train_supervised(input="train.txt", lr=0.0001, epoch=10000, wordNgrams=1, bucket=200000, dim=400, loss='hs')

In [91]:
model_2 = fasttext.train_supervised(input="gas.1990.train", autotuneValidationFile='gas.1990.valid', autotuneDuration=300)

In [92]:
model_2.save_model("model.gas.1990")

In [96]:
model_2.test('gas.1990.valid')

(68, 0.8970588235294118, 0.8970588235294118)

### Predict

In [74]:
list_preds = []
a = 0
for i, r in test.iterrows():
    print(row["text_clean"])
    #temp = model.predict(row["text_clean"])
    a = a + 1
    if a > 5: break
    list_preds.append(temp)

het van organiseert dit jaar de landelijke dagen bij de in vanmiddag wijden de honderd deelnemers zich aan het het is de bedoeling dat dit spel de de verbetert het weekend trekken de deelnemers naar waar de vaardigheden om te werken worden getraind
het van organiseert dit jaar de landelijke dagen bij de in vanmiddag wijden de honderd deelnemers zich aan het het is de bedoeling dat dit spel de de verbetert het weekend trekken de deelnemers naar waar de vaardigheden om te werken worden getraind
het van organiseert dit jaar de landelijke dagen bij de in vanmiddag wijden de honderd deelnemers zich aan het het is de bedoeling dat dit spel de de verbetert het weekend trekken de deelnemers naar waar de vaardigheden om te werken worden getraind
het van organiseert dit jaar de landelijke dagen bij de in vanmiddag wijden de honderd deelnemers zich aan het het is de bedoeling dat dit spel de de verbetert het weekend trekken de deelnemers naar waar de vaardigheden om te werken worden getraind
het 

In [67]:
list_preds

[(('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',), array([0.67702383])),
 (('__label__0',

In [58]:
test["label"]

19       ((__label__0,), [0.6770238280296326])
37       ((__label__0,), [0.6770238280296326])
133      ((__label__0,), [0.6770238280296326])
141      ((__label__0,), [0.6770238280296326])
172      ((__label__0,), [0.6770238280296326])
                         ...                  
42811    ((__label__0,), [0.6770238280296326])
42812    ((__label__0,), [0.6770238280296326])
42813    ((__label__0,), [0.6770238280296326])
42814    ((__label__0,), [0.6770238280296326])
42815    ((__label__0,), [0.6770238280296326])
Name: label, Length: 41350, dtype: object

In [66]:
i = 900
print(test["text"][i])
print(model.predict(test["text_clean"][i]))
#print(test.iloc[3,1].values)

• V^y energiebesparing! doet u er iets aan?^S^//| _^_^_H_H_te m^l^L^Ê WÊL^ Tei< onderstaande bedrijven helpen u graag op weg. V*"^=__^ £ ' ft _— -------- . ,V1U BRUNSSUM' 9e; BRUNSSUM: ___ri^_i__riV llWllTi 11 Als uw woning gemiddeld 2.000 m 3 aard- ou . ]*•» _r A _■ Al I V kil '•' — _r W . ■ I ■ I I _ w I _ I F__l_P_F TJJ -J.\! r_-J T 3 5 Installatiebedrijf I____&_m_l lU gas per jaar nodig heeft voor verwarming, _»ftTM.C7:Vg»I H ■ vilij w* A. Hermse b.v. Frans Goeman ° 'Mll [Il ij f 111 ■■ il 1 l*J dan kiest u 00k voor de nieuwe Landelijk erkend install^ Ï;J Pr. Hendriklaan 172 _W i ■ lil lw II fli _I k I lw I B^ 1 voor gas, water en sanitair 15.; __J_ii U_____l ____________J_l___l Thermocompact Low NOx CV-ketel van Horizonstraat 55 i«j ■,'I _TiT_ _',<_■ il I ll_!_BT»i wv —A'V'l Br1»^! -x '6i I __LJJ L^^^^^^^^3_^__^_^_^__l_T«__^__^__id____^__i^__A__^__| I Vaillant: » 17. CADIER EN KEER: DE KAMPIOEN BIJ EEN GEMIDDELD GAS- HEERLEN: ie. WSSt rr Installatiebedrijf \\ i^P %?n! verbruik van